<a href="https://colab.research.google.com/github/jbarriosmuriel/btc_data/blob/main/Financial_Data_Structures_(Bars).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Original Article:

Installing and Importing Requisite Packages


In [ ]:
pip install shrimpy-python

     |████████████████████████████████| 81kB 4.7MB/s 


In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import math
import plotly
import shrimpy
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Collecting the Data


In [ ]:
#collecting the historical candlestick data
shrimpy_public_key ='...'
shrimpy_secret_key = '...'
client = shrimpy.ShrimpyApiClient(shrimpy_public_key, shrimpy_secret_key)
candles = client.get_candles(
    'coinbasepro', # exchange
    'BTC',     # base_trading_symbol
    'USD',     # quote_trading_symbol
    '1m'       # interval
)

time = []
open = []
high = []
low = []
close = []
vol =[]

#formatting the data to match the plotting library
for candle in candles:
    time.append(candle['time'])
    open.append(candle['open'])
    high.append(candle['high'])
    low.append(candle['low'])
    close.append(candle['close'])
    vol.append(candle['volume'])

#plotting the candlesticks
fig = go.Figure(data=[go.Candlestick(x=time,
                       open=open, high=high,
                       low=low, close=close)])
fig.show()

Formatting Data

In [ ]:
#create dataframe from lists
df = pd.DataFrame(
    {'time': [datetime.strptime(x,"%Y-%m-%dT%H:%M:%S.%fZ") for x in time], #formatting our timestamp
     'open': [float(x.strip('0')) for x in open], #removing trailing 0's and converting to float
     'high': [float(x.strip('0')) for x in high],
     'low': [float(x.strip('0')) for x in low],
     'close': [float(x.strip('0')) for x in close],
     'vol': [float(x.strip('0')) for x in vol]
    })

#create dictionary from dataframe
df_dict = df.to_dict('records')

Defining Dollar Bar Function


In [ ]:
def get_dollar_bars(time_bars, dollar_threshold):

    # initialize an empty list of dollar bars
    dollar_bars = []

    # initialize the running dollar volume at zero
    running_volume = 0

    # initialize the running high and low with placeholder values
    running_high, running_low = 0, math.inf

    # for each time bar...
    for i in range(len(time_bars)):

        # get the timestamp, open, high, low, close, and volume of the next bar
        next_close, next_high, next_low, next_open, next_timestamp, next_volume = [time_bars[i][k] for k in ['close', 'high', 'low', 'open', 'time', 'vol']]

        # get the midpoint price of the next bar (the average of the open and the close)
        midpoint_price = ((next_open) + (next_close))/2

        # get the approximate dollar volume of the bar using the volume and the midpoint price
        dollar_volume = next_volume * midpoint_price

        # update the running high and low
        running_high, running_low = max(running_high, next_high), min(running_low, next_low)

        # if the next bar's dollar volume would take us over the threshold...
        if dollar_volume + running_volume >= dollar_threshold:

            # set the timestamp for the dollar bar as the timestamp at which the bar closed (i.e. one minute after the timestamp of the last minutely bar included in the dollar bar)
            bar_timestamp = next_timestamp + timedelta(minutes=1)

            # add a new dollar bar to the list of dollar bars with the timestamp, running high/low, and next close
            dollar_bars += [{'timestamp': bar_timestamp, 'open': next_open, 'high': running_high, 'low': running_low, 'close': next_close}]

            # reset the running volume to zero
            running_volume = 0

            # reset the running high and low to placeholder values
            running_high, running_low = 0, math.inf

        # otherwise, increment the running volume
        else:
            running_volume += dollar_volume

    # return the list of dollar bars
    return dollar_bars

Creating Dollar Bars

In [ ]:
dollar_bars = get_dollar_bars(df_dict, 5000000) #5,000,000 is an arbitrarily selected threshold
df = pd.DataFrame(dollar_bars)
df.head()

,timestamp,open,high,low,close
0,2020-12-17 04:36:00,21856.52,21913.44,21817.07,21828.86
1,2020-12-17 04:54:00,21820.45,21876.06,21784.55,21824.54
2,2020-12-17 05:07:00,21842.77,21891.68,21800.00,21865.59
3,2020-12-17 05:19:00,21987.50,22004.64,21857.38,21992.26
4,2020-12-17 05:31:00,21994.19,22013.31,21961.70,21981.49


In [ ]:
fig = go.Figure(data=[go.Candlestick(x=df.timestamp,
                     open=df.open, high=df.high,
                     low=df.low, close=df.close)])
fig.show()

Plotting both bars together


In [ ]:
trace1 = go.Candlestick(x=time,
                       open=open, high=high,
                       low=low, close=close,
                       name = 'time_bars'
)

trace2 = go.Candlestick(x=df.timestamp,
                       open=df.open, high=df.high,
                       low=df.low, close=df.close,
                       name = 'dollar_bars',
                       increasing_line_color= 'cyan', decreasing_line_color= 'black'

)

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1)
fig.add_trace(trace2,secondary_y=True)


fig.show()